In [19]:
import requests, json
import pandas as pd
import numpy as np
from pprint import pprint
import requests



In [45]:
print(os.getcwd())

/Users/trajche/Desktop/python/FPL_Data_Pull


In [47]:
#logging in and getting my team data

s = requests.Session()

headers = {
   'authority': 'users.premierleague.com' ,
   'cache-control': 'max-age=0' ,
   'upgrade-insecure-requests': '1' ,
   'origin': 'https://fantasy.premierleague.com' ,
   'content-type': 'application/x-www-form-urlencoded' ,
   'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36' ,
   'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9' ,
   'sec-fetch-site': 'same-site' ,
   'sec-fetch-mode': 'navigate' ,
   'sec-fetch-user': '?1' ,
   'sec-fetch-dest': 'document' ,
   'referer': 'https://fantasy.premierleague.com/my-team' ,
   'accept-language': 'en-US,en;q=0.9,he;q=0.8' ,
}
with open("/Users/trajche/Desktop/python/FPL_Data_Pull/credentials.json", 'r') as f:
  data = json.load(f)


url = "https://users.premierleague.com/accounts/login/"

res = s.post(url, data = data,  headers = headers)

team_url = "https://fantasy.premierleague.com/api/my-team/203128/"
res = s.get(team_url)
json.loads(res.content)


{'picks': [{'element': 113,
   'position': 1,
   'selling_price': 45,
   'multiplier': 1,
   'purchase_price': 45,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 285,
   'position': 2,
   'selling_price': 75,
   'multiplier': 1,
   'purchase_price': 75,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 306,
   'position': 3,
   'selling_price': 70,
   'multiplier': 1,
   'purchase_price': 70,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 284,
   'position': 4,
   'selling_price': 70,
   'multiplier': 1,
   'purchase_price': 70,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 146,
   'position': 5,
   'selling_price': 60,
   'multiplier': 1,
   'purchase_price': 60,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 283,
   'position': 6,
   'selling_price': 130,
   'multiplier': 2,
   'purchase_price': 130,
   'is_captain': True,
   'is_vice_captain': False},
  {'element': 374,
   'positio

In [ ]:
s = json.loads(res.content)

In [39]:

myteam = pd.json_normalize(s['picks'])
print(myteam.head(5))

   element  position  selling_price  multiplier  purchase_price  is_captain  \
0      113         1             45           1              45       False   
1      285         2             75           1              75       False   
2      306         3             70           1              70       False   
3      284         4             70           1              70       False   
4      146         5             60           1              60       False   

   is_vice_captain  
0            False  
1            False  
2            False  
3            False  
4            False  


In [2]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 4657517}


In [4]:

players = r['elements']
print(players[0)]

{'chance_of_playing_next_round': None, 'chance_of_playing_this_round': None, 'code': 58822, 'cost_change_event': 0, 'cost_change_event_fall': 0, 'cost_change_start': 0, 'cost_change_start_fall': 0, 'dreamteam_count': 0, 'element_type': 2, 'ep_next': '2.3', 'ep_this': None, 'event_points': 0, 'first_name': 'Cédric', 'form': '0.0', 'id': 1, 'in_dreamteam': False, 'news': '', 'news_added': None, 'now_cost': 45, 'photo': '58822.jpg', 'points_per_game': '2.3', 'second_name': 'Alves Soares', 'selected_by_percent': '0.3', 'special': False, 'squad_number': None, 'status': 'a', 'team': 1, 'team_code': 3, 'total_points': 48, 'transfers_in': 0, 'transfers_in_event': 0, 'transfers_out': 0, 'transfers_out_event': 0, 'value_form': '0.0', 'value_season': '10.7', 'web_name': 'Cédric', 'minutes': 1481, 'goals_scored': 1, 'assists': 1, 'clean_sheets': 3, 'goals_conceded': 27, 'own_goals': 0, 'penalties_saved': 0, 'penalties_missed': 0, 'yellow_cards': 3, 'red_cards': 0, 'saves': 0, 'bonus': 3, 'bps': 29

In [10]:
players = pd.json_normalize(r['elements'])

p1 = players[['id', 'web_name', 'team', 'element_type', 'now_cost']].head()

In [12]:
p1['game_cost'] = p1['now_cost'] / 10
print(p1.head())

   id web_name  team  element_type  now_cost  game_cost
0   1   Cédric     1             2        45        4.5
1   2     Leno     1             1        45        4.5
2   3    Xhaka     1             3        50        5.0
3   4   Elneny     1             3        45        4.5
4   5  Holding     1             2        45        4.5


In [28]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'entry/203128/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'current_event': None,
  'favourite_team': 12,
  'id': 203128,
  'joined_time': '2022-07-05T18:18:24.725376Z',
  'kit': None,
  'last_deadline_bank': None,
  'last_deadline_total_transfers': 0,
  'last_deadline_value': None,
  'leagues': {...},
  'name': 'van Trajk',
  'name_change_blocked': False,
  'player_first_name': 'Andrej',
  'player_last_name': 'Trajkovski',
  'player_region_id': 241,
  'player_region_iso_code_long': 'ENG',
  'player_region_iso_code_short': 'EN',
  'player_region_name': 'England',
  'started_event': 1,
  'summary_event_points': None,
  'summary_event_rank': None,
  'summary_overall_points': None,
  'summary_overall_rank': None}


In [32]:
#Get competitors teams

base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'my-team/203128/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)


{'detail': 'Authentication credentials were not provided.'}
